In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.sparse import coo_matrix
# ---------------------------------------------------------
# Path 설정
# ---------------------------------------------------------

BASE_DIR = Path().resolve().parent.parent.parent
# base_path =  BASE_DIR / "data" / "train" / "TL_csv"
base_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "1.여행로그 장소 추천 고도화" / "data"
save_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "1.여행로그 장소 추천 고도화" / "model"


In [18]:


# ========== STEP 1: Load CSV ==========

user_df = pd.read_csv(base_path / "rating_user_info_all.csv")
item_df = pd.read_csv(base_path / "rating_item_info_all_sido.csv")

print("=== Raw Data Shapes ===")
print("user_df:", user_df.shape)
print("item_df:", item_df.shape)

# ========== STEP 2: Remove unnecessary columns ==========
if "Unnamed: 0" in user_df.columns:
    print('unnamed in user_df')
    user_df = user_df.drop(columns=["Unnamed: 0"])

if "Unnamed: 0" in item_df.columns:
    print('unnamed in item_df')
    item_df = item_df.drop(columns=["Unnamed: 0"])

# ========== STEP 3: Fix item_df rating + SIDO ==========
item_df["ratings"] = pd.to_numeric(item_df["ratings"], errors="coerce")

item_df["SIDO"] = item_df["SIDO"].replace("", np.nan)
item_df["SIDO"] = item_df["SIDO"].fillna("Unknown")

# ========== STEP 4: Check key columns ==========
print("\n=== Column Check ===")
print("user_df columns:", user_df.columns.tolist())
print("item_df columns:", item_df.columns.tolist())

# ========== STEP 5: Basic integrity check ==========
print("\nUnique TRAVELER_IDs:")
print("user_df:", user_df["TRAVELER_ID"].nunique())
print("item_df:", item_df["TRAVELER_ID"].nunique())

print("\n=== Sample Rows ===")
print(user_df.head())
print(item_df.head())


=== Raw Data Shapes ===
user_df: (2880, 17)
item_df: (11115, 6)
unnamed in user_df
unnamed in item_df

=== Column Check ===
user_df columns: ['TRAVELER_ID', 'GENDER', 'AGE_GRP', 'MARR_STTS', 'JOB_NM', 'INCOME', 'TRAVEL_NUM', 'TRAVEL_STYL_1', 'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 'MONTH', 'SEASON', 'HOW_LONG']
item_df columns: ['TRAVELER_ID', 'VISIT_AREA_NM', 'ratings', 'SIDO', 'VISIT_AREA_TYPE_CD']

Unique TRAVELER_IDs:
user_df: 2880
item_df: 2578

=== Sample Rows ===
  TRAVELER_ID GENDER  AGE_GRP  MARR_STTS  JOB_NM  INCOME  TRAVEL_NUM  \
0     g003414      남       20          1       3       1           2   
1     h006468      남       40          2       1       8           2   
2     h003272      여       50          1       2       4           1   
3     h006175      여       20          1       2       6           2   
4     e000899      여       30          1       2       5           1   

   TR

In [ ]:


# === STEP 2: Create LightFM user_features and item_features ===

# -------------------------------------
# 1) Train users = users who appear in item_df
# -------------------------------------
train_user_ids = item_df["TRAVELER_ID"].unique()

train_user_df = user_df[user_df["TRAVELER_ID"].isin(train_user_ids)].copy()

# 사용자 id를 LightFM 정수 id로 매핑
train_user_df["user_id"] = train_user_df["TRAVELER_ID"].astype("category").cat.codes

num_users = train_user_df["user_id"].max() + 1


# -------------------------------------
# 2) USER FEATURES (for cold start)
# -------------------------------------

# 사용할 특징 선택 (확장 가능)
uf = train_user_df[[
    "user_id",
    "GENDER",
    "AGE_GRP",
    "MARR_STTS",
    "JOB_NM",
    "INCOME",
    "TRAVEL_NUM",
    "TRAVEL_STYL_1",
    "TRAVEL_STATUS_RESIDENCE",
    "TRAVEL_STATUS_DESTINATION",
    "TRAVEL_STATUS_ACCOMPANY",
    "TRAVEL_MOTIVE_1",
    "TRAVEL_COMPANIONS_NUM",
    "MONTH",
    "SEASON",
    "HOW_LONG"
]]

# One-hot encoding
gender = pd.get_dummies(uf["GENDER"], prefix="gender")
age = pd.get_dummies(uf["AGE_GRP"], prefix="age_grp")
marr = pd.get_dummies(uf["MARR_STTS"], prefix="marr_stts")
job = pd.get_dummies(uf["JOB_NM"], prefix="job_nm")
income = pd.get_dummies(uf["INCOME"], prefix="income")
travel_num = pd.get_dummies(uf["TRAVEL_NUM"], prefix="travel_num")
travel_styl_1 = pd.get_dummies(uf["TRAVEL_STYL_1"], prefix="travel_styl_1")
travel_status_residence = pd.get_dummies(uf["TRAVEL_STATUS_RESIDENCE"], prefix="travel_status_residence")
travel_status_destination = pd.get_dummies(uf["TRAVEL_STATUS_DESTINATION"], prefix="travel_status_destination")
travel_status_accompany = pd.get_dummies(uf["TRAVEL_STATUS_ACCOMPANY"], prefix="travel_status_accompany")
travel_motive_1 = pd.get_dummies(uf["TRAVEL_MOTIVE_1"], prefix="travel_motive_1")
travel_companions_num = pd.get_dummies(uf["TRAVEL_COMPANIONS_NUM"], prefix="travel_companions_num")
month = pd.get_dummies(uf["MONTH"], prefix="month")
season = pd.get_dummies(uf["SEASON"], prefix="season")
how_long = pd.get_dummies(uf["HOW_LONG"], prefix="how_long")

user_features_df = pd.concat([
    uf["user_id"],
    gender, age, marr, job, income, travel_num, travel_styl_1,
    travel_status_residence, travel_status_destination, travel_status_accompany, 
    travel_motive_1, travel_companions_num, month, season, how_long
], axis=1)

# user_id 기준 정렬
user_features_df = user_features_df.sort_values("user_id")

# user_id 컬럼을 제외하고 sparse로 변환
user_features_matrix = coo_matrix(user_features_df.drop(columns=["user_id"]).values)

print("user_features_matrix shape:", user_features_matrix.shape)





user_features_matrix shape: (2578, 144)


In [78]:
# -------------------------------------
# 3) ITEM FEATURES
# -------------------------------------

tmp_item = item_df[["VISIT_AREA_NM", "SIDO", "VISIT_AREA_TYPE_CD"]].copy()
tmp_item["item_id"] = tmp_item["VISIT_AREA_NM"].astype("category").cat.codes
item_df["item_id"] = item_df["VISIT_AREA_NM"].astype("category").cat.codes

# One-hot encoding for item attributes
sido = pd.get_dummies(tmp_item["SIDO"], prefix="sido")
itype = pd.get_dummies(tmp_item["VISIT_AREA_TYPE_CD"], prefix="itype")

item_features_df = pd.concat([
    tmp_item["item_id"], sido, itype
], axis=1)

# item_id별로 하나의 row만 남기기
item_features_df = item_features_df.groupby("item_id").first().reset_index()

item_features_df = item_features_df.sort_values("item_id")

item_features_matrix = coo_matrix(item_features_df.drop(columns=["item_id"]).values)

num_items = item_features_df["item_id"].max() + 1

print("item_features_matrix shape:", item_features_matrix.shape)


item_features_matrix shape: (1257, 26)


item_features_matrix shape: (1141, 26)
user_features_matrix shape: (2275, 143)

In [64]:
from lightfm import LightFM
from lightfm.evaluation import auc_score

# ============================================
# 1) interaction_df 만들기 (user-item-rating)
# ============================================
interaction_df = item_df[["TRAVELER_ID", "VISIT_AREA_NM", "ratings"]].copy()

# TRAVELER_ID -> user_id 매핑
interaction_df["user_id"] = interaction_df["TRAVELER_ID"].astype("category").cat.codes
interaction_df["item_id"] = interaction_df["VISIT_AREA_NM"].astype("category").cat.codes
# 1) category 생성 (공통 매핑)
user_cat = interaction_df["TRAVELER_ID"].astype("category")


user_df["user_id"] = user_df["TRAVELER_ID"].map(
    dict(zip(user_cat, user_cat.cat.codes))
)

num_users = int(interaction_df["user_id"].max() + 1)
num_items = int(interaction_df["item_id"].max() + 1)

# 전체 데이터를 "미래까지 포함된 interaction"이라고 가정
full_interactions = coo_matrix(
    (interaction_df["ratings"].astype(float),
     (interaction_df["user_id"], interaction_df["item_id"])),
    shape=(num_users, num_items)
)

print("num_users:", num_users, "num_items:", num_items)
print("full_interactions nnz:", full_interactions.nnz)

num_users: 2578 num_items: 1257
full_interactions nnz: 11115


In [38]:
# ============================================
# 2) Cold-start 유저 선정 (완전 신규 유저 가정)
# ============================================
rng = np.random.default_rng(42)
all_user_ids = interaction_df["user_id"].unique()
valid_test_user_ids = rng.choice(
    all_user_ids,
    size=int(len(all_user_ids) * 0.2),  # 20%를 valid test 유저로 가정
    replace=False
)
test_user_ids = rng.choice(
    valid_test_user_ids,
    size=int(len(valid_test_user_ids) * 0.5),  # 10%를 신규(test) 유저로 가정
    replace=False
)

# train 에서는 cold user 의 interaction 전부 제거
train_df = interaction_df[~interaction_df["user_id"].isin(valid_test_user_ids)].copy()

train_interactions = coo_matrix(
    (train_df["ratings"].astype(float),
     (train_df["user_id"], train_df["item_id"])),
    shape=(num_users, num_items)
)

# cold users only
valid_test_df = interaction_df[interaction_df["user_id"].isin(valid_test_user_ids)]

valid_df = valid_test_df[~valid_test_df["user_id"].isin(test_user_ids)].copy()
test_df = valid_test_df[valid_test_df["user_id"].isin(test_user_ids)]

valid_interactions = coo_matrix(
    (valid_df["ratings"].astype(float), (valid_df["user_id"], valid_df["item_id"])),
    shape=(num_users, num_items)
)
test_interactions = coo_matrix(
    (test_df["ratings"].astype(float), (test_df["user_id"], test_df["item_id"])),
    shape=(num_users, num_items)
)
print("train nnz:", train_interactions.nnz)
print("valid nnz:", valid_interactions.nnz)
print("test nnz:", test_interactions.nnz)

train nnz: 8952
valid nnz: 1108
test nnz: 1055


In [39]:
train_interactions = train_interactions.tocsr()
valid_interactions = valid_interactions.tocsr()
test_interactions = test_interactions.tocsr()
user_features_matrix = user_features_matrix.tocsr()
item_features_matrix = item_features_matrix.tocsr()

# 2) dtype 고정
train_interactions = train_interactions.astype(np.float32)
valid_interactions = valid_interactions.astype(np.float32)
test_interactions = test_interactions.astype(np.float32)
user_features_matrix = user_features_matrix.astype(np.float32)
item_features_matrix = item_features_matrix.astype(np.float32)

In [40]:
# ============================================
# 3) LightFM 학습 (user/item feature 사용)
# ============================================
model = LightFM(no_components=64, loss='warp')


In [41]:
model.fit(
    train_interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    epochs=20,
    num_threads=4
)

In [11]:
len(all_user_auc)

515

In [12]:
cold_auc = all_user_auc.mean()
print("Cold-start user AUC:", cold_auc)

Cold-start user AUC: 0.8221924


In [ ]:
# ---------------------------------------------------------
# Inference에 쓸 Model, Data 저장
# ---------------------------------------------------------

In [15]:
import pickle

with open(save_path / "lightfm_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [16]:
import pickle

with open(save_path / "lightfm_model.pkl", "rb") as f:
    model = pickle.load(f)

In [81]:
from scipy.sparse import save_npz
save_npz(save_path / "item_features_matrix.npz", item_features_matrix)

In [83]:
item_meta = (
    item_df[
        ["item_id", "VISIT_AREA_NM", "SIDO", "VISIT_AREA_TYPE_CD", "ratings"]
    ]
    .drop_duplicates(subset=["item_id"])
)
item_meta.to_csv(save_path / "item_meta.csv", index=False)

In [84]:
import json
# 4) user feature 컬럼 순서 저장 (user_id 제외)
user_feature_cols = user_features_df.drop(columns=["user_id"]).columns.tolist()

with open(save_path / "user_feature_cols.json", "w", encoding="utf-8") as f:
    json.dump(user_feature_cols, f, ensure_ascii=False, indent=2)

# 5) (선택) VISIT_AREA_NM -> item_id 매핑 저장 (편의용)
name_to_item_id = dict(zip(item_meta["VISIT_AREA_NM"], item_meta["item_id"]))

with open(save_path / "name_to_item_id.pkl", "wb") as f:
    pickle.dump(name_to_item_id, f)

In [51]:
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k

# train 기준 AUC (참고용)
train_auc = auc_score(
    model,
    train_interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    num_threads=4
).mean()

# valid AUC (train에서 본 아이템 제외하고 평가)
valid_auc = auc_score(
    model,
    valid_interactions,
    train_interactions=train_interactions,   # 중요!
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    num_threads=4
).mean()

# precision@k, recall@k on valid
k = 10

valid_precision = precision_at_k(
    model,
    valid_interactions,
    train_interactions=train_interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    k=k,
    num_threads=4
).mean()

valid_recall = recall_at_k(
    model,
    valid_interactions,
    train_interactions=train_interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    k=k,
    num_threads=4
).mean()

print(f"train AUC      : {train_auc:.4f}")
print(f"valid AUC      : {valid_auc:.4f}")
print(f"valid P@{k}    : {valid_precision:.4f}")
print(f"valid R@{k}    : {valid_recall:.4f}")


train AUC      : 0.8318
valid AUC      : 0.8179
valid P@10    : 0.0136
valid R@10    : 0.0620


In [52]:
# 일단 이미 학습된 model로 test 메트릭만 보고 싶다면:
test_auc = auc_score(
    model,
    test_interactions,
    train_interactions=train_interactions,  # train + valid를 train으로 간주
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    num_threads=4
).mean()

test_precision = precision_at_k(
    model,
    test_interactions,
    train_interactions=train_interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    k=k,
    num_threads=4
).mean()

test_recall = recall_at_k(
    model,
    test_interactions,
    train_interactions=train_interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    k=k,
    num_threads=4
).mean()

print(f"test AUC    : {test_auc:.4f}")
print(f"test P@{k}  : {test_precision:.4f}")
print(f"test R@{k}  : {test_recall:.4f}")


test AUC    : 0.8213
test P@10  : 0.0062
test R@10  : 0.0306


In [45]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# test를 COO로 바꿈
test_coo = test_interactions.tocoo()

true_y = test_coo.data              # 실제 rating
user_ids = test_coo.row
item_ids = test_coo.col

# 예측값
pred_y = model.predict(
    user_ids,
    item_ids,
    user_features=user_features_matrix,
    item_features=item_features_matrix
)

rmse = mean_squared_error(true_y, pred_y)
mae = mean_absolute_error(true_y, pred_y)

print("Test RMSE:", rmse)
print("Test MAE :", mae)


Test RMSE: 94887.9609375
Test MAE : 306.42449951171875


In [58]:
# === 2) cold-start 유저에게 추천 아이템 뽑기 ===
# test_user_ids: 위에서 이미 뽑아둔 신규 유저 id 배열 (train에 없음)

# (1) item_id -> VISIT_AREA_NM 역매핑 준비
item_id_to_name = (
    interaction_df[["item_id", "VISIT_AREA_NM"]]
    .drop_duplicates()
    .set_index("item_id")["VISIT_AREA_NM"]
)

def recommend_for_cold_user(
    model,
    user_id: int,
    user_features,
    item_features,
    num_items: int,
    topn: int = 10
):
    """
    과거 평점 없이(user-item interaction 없이)
    user_features만으로 추천하는 cold-start 추천 함수.
    """
    # 0 ~ num_items-1 전체에 대해 점수 예측
    item_ids = np.arange(num_items, dtype=np.int32)

    scores = model.predict(
        user_ids=user_id,
        item_ids=item_ids,
        user_features=user_features,
        item_features=item_features,
        num_threads=4
    )

    # 높은 점수 순으로 상위 topn 인덱스
    top_indices = np.argsort(-scores)[:topn]

    return top_indices, scores[top_indices]


# (2) 예시: 임의의 cold-start 유저 하나 뽑아서 추천
example_user = int(test_user_ids[0])
top_indices, top_scores = recommend_for_cold_user(
    model,
    user_id=example_user,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    num_items=num_items,
    topn=10
)

print(f"\n[추천 결과] cold-start user_id = {example_user}")
for rank, (item_idx, score) in enumerate(zip(top_indices, top_scores), start=1):
    item_name = item_id_to_name.loc[item_idx]
    print(f"{rank:2d}. item_id={item_idx:4d}, score={score:.4f}, place={item_name}")



[추천 결과] cold-start user_id = 449
 1. item_id= 264, score=-277.0304, place=두 모 몽돌 해수욕장
 2. item_id= 793, score=-277.0304, place=와현모래숲해수욕장
 3. item_id= 575, score=-277.0304, place=소매물도
 4. item_id= 736, score=-277.0304, place=연화도
 5. item_id= 259, score=-277.0304, place=동정호
 6. item_id= 459, score=-277.0304, place=사량도 대항해수욕장
 7. item_id= 667, score=-277.0304, place=쑥섬
 8. item_id=1109, score=-277.0304, place=통영 케이블카
 9. item_id= 333, score=-277.0304, place=망치몽돌해수욕장
10. item_id= 648, score=-277.0304, place=신선대


In [59]:
def recommend_with_info(
    model,
    user_id: int,
    user_features,
    item_features,
    item_df,
    num_items: int,
    topn: int = 10
):
    # 전체 아이템 점수 계산
    item_ids = np.arange(num_items)
    scores = model.predict(
        user_ids=user_id,
        item_ids=item_ids,
        user_features=user_features,
        item_features=item_features,
        num_threads=4
    )

    # 상위 topn 인덱스 추출
    top_idx = np.argsort(-scores)[:topn]
    top_scores = scores[top_idx]

    # item_df에서 item 정보 가져오기
    # 1) VISIT_AREA_NM → item_id 매핑 만들기
    tmp_mapping = (
        item_df[["VISIT_AREA_NM"]]
        .drop_duplicates()
    )
    tmp_mapping["item_id"] = tmp_mapping["VISIT_AREA_NM"].astype("category").cat.codes

    # 2) 추천된 item_id와 item_df JOIN
    result = pd.DataFrame({"item_id": top_idx, "score": top_scores})
    result = result.merge(tmp_mapping, on="item_id", how="left")
    result = result.merge(
        item_df.drop_duplicates(subset=["VISIT_AREA_NM"]),
        on="VISIT_AREA_NM",
        how="left"
    )

    # 보기 좋게 정리
    cols = ["rank", "item_id", "score", "VISIT_AREA_NM", "SIDO", "VISIT_AREA_TYPE_CD", "ratings"]
    result = result.assign(rank=np.arange(1, len(result) + 1))[cols]

    return result


In [60]:
example_user = int(test_user_ids[0])

recommend_df = recommend_with_info(
    model,
    user_id=example_user,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    item_df=item_df,
    num_items=num_items,
    topn=10
)

print(recommend_df)


   rank  item_id       score VISIT_AREA_NM SIDO  VISIT_AREA_TYPE_CD   ratings
0     1      264 -277.030396   두 모 몽돌 해수욕장   경남                   1  3.000000
1     2      793 -277.030396     와현모래숲해수욕장   경남                   1  3.000000
2     3      575 -277.030396          소매물도   경남                   1  5.000000
3     4      736 -277.030396           연화도   경남                   1  5.000000
4     5      259 -277.030396           동정호   경남                   1  4.000000
5     6      459 -277.030396    사량도 대항해수욕장   경남                   1  5.000000
6     7      667 -277.030396            쑥섬   경남                   1  4.000000
7     8     1109 -277.030396       통영 케이블카   경남                   1  4.000000
8     9      333 -277.030396      망치몽돌해수욕장   경남                   1  4.000000
9    10      648 -277.030396           신선대   경남                   1  3.333333


In [65]:
# user_df 에 user_id가 있다면 이 코드 사용
jeju_users = user_df[
    (user_df["TRAVEL_STATUS_DESTINATION"] == "제주")
    & (user_df["user_id"].isin(test_user_ids))
]

jeju_test_interactions = test_df[
    test_df["user_id"].isin(jeju_users["user_id"])
].copy()

print(jeju_test_interactions.head())


     TRAVELER_ID VISIT_AREA_NM   ratings  user_id  item_id
1835     g005682    아르떼 뮤지엄 제주  4.333333      851      674
1836     g005682         천지연폭포  3.000000      851     1054
1837     g005682         천제연폭포  5.000000      851     1052
1838     g005682        제주동문시장  4.000000      851      983
1839     g005682         바이 제주  3.000000      851      389


In [70]:
def recommend_for_cold_user_with_info(
    model,
    user_id,
    user_features,
    item_features,
    item_df,
    num_items,
    topn=10
):
    # 1) 전체 아이템 점수 예측
    item_ids = np.arange(num_items)
    scores = model.predict(
        user_ids=user_id,
        item_ids=item_ids,
        user_features=user_features,
        item_features=item_features,
        num_threads=4
    )
    
    # 2) 상위 topN item 선택
    top_idx = np.argsort(-scores)[:topn]
    top_scores = scores[top_idx]

    # 3) item_df에서 item info 조인
    result = pd.DataFrame({
        "item_id": top_idx,
        "score": top_scores
    })

    result = result.merge(
        item_df,#.drop_duplicates(subset=["item_id"]),
        on="item_id",
        how="left"
    )

    # 보기 좋게 정렬
    result["rank"] = result["score"].rank(ascending=False).astype(int)
    result = result.sort_values("rank")

    return result[
        ["rank", "item_id", "score", "VISIT_AREA_NM", "SIDO", "VISIT_AREA_TYPE_CD", "ratings"]
    ]


In [73]:
example_user = int(jeju_users.iloc[0]["user_id"])

recommend_df = recommend_with_info(
    model,
    user_id=example_user,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    item_df=item_df,
    num_items=num_items,
    topn=10
)

print(recommend_df)

   rank  item_id       score VISIT_AREA_NM     SIDO  VISIT_AREA_TYPE_CD  \
0     1        0 -266.772095       11고지 습지  제주특별자치도                   1   
1     2      947 -266.772095    제주 밭담 테마공원  제주특별자치도                   1   
2     3      599 -266.772095        쇠머리 오름  제주특별자치도                   1   
3     4      600 -266.772095           쇠소깍  제주특별자치도                   1   
4     5      602 -266.772095   쇠소깍 산물 관광농원  제주특별자치도                   1   
5     6      344 -266.772095          모슬포항  제주특별자치도                   1   
6     7      609 -266.772095           수산동  제주특별자치도                   1   
7     8      184 -266.772095          다락쉼터  제주특별자치도                   1   
8     9      185 -266.772095        다랑쉬 오름  제주특별자치도                   1   
9    10      937 -266.772095   제주 곶자왈 도립공원  제주특별자치도                   1   

    ratings  
0  5.000000  
1  3.333333  
2  5.000000  
3  4.333333  
4  5.000000  
5  2.333333  
6  3.333333  
7  5.000000  
8  4.333333  
9  3.333333  
